# Network of Thrones
### The Weighted Edges of the Storm of Swords Network

A. Beveridge and J. Shan, "Network of Thrones," Math Horizons Magazine , Vol. 23, No. 4 (2016), pp. 18-22.
https://www.macalester.edu/~abeverid/thrones.html

Images courtesy of the Game of Thrones Wiki.

Leverages SAND, py2cytoscape, and igraph to implement the following features:

* An edge's thickness represents its weight.
* The color of a vertex's border indicates its community.
* The size of a vertex corresponds to its PageRank value.
* The size of its label corresponds to its betweenness centrality.
* Degree-sorted circle layout based on group membership, with saved and reloadable manual positions.

## Network Analysis

In [1]:
from py2cytoscape.data.cynetwork import CyNetwork
from py2cytoscape.data.cyrest_client import CyRestClient
from py2cytoscape.data.style import StyleUtil as su
import py2cytoscape.util.cytoscapejs as cyjs
import py2cytoscape.cytoscapejs as renderer

from IPython.display import Image

import igraph as igraph

from sand.csv import csv_to_dicts
import sand.graph as graph
import sand.cytoscape.positions as scp
import sand.cytoscape.app as app
from sand.cytoscape.themes import ops, colors
import sand.groups as groups

<IPython.core.display.Javascript object>

In [2]:
vertex_data = csv_to_dicts('./data/storm_of_swords_vertices.csv')

In [3]:
edge_data = csv_to_dicts('./data/storm_of_swords_edges.csv')

In [4]:
g = graph.from_vertices_and_edges(
                    vertices=vertex_data, 
                    edges=edge_data, 
                    vertex_name_key='name', 
                    vertex_id_key='name', 
                    edge_foreign_keys=('source_v', 'target_v'),
                    directed=False)
g.summary()

'IGRAPH UNW- 107 352 -- \n+ attr: group (v), imageurl (v), indegree (v), label (v), name (v), outdegree (v), wiki (v), source_v (e), target_v (e), weight (e)'

In [5]:
# Ensure weight doesn't get added as a string
g.es['weight'] = list(map(lambda x: int(x), g.es['weight']))

### Community Detection / Clustering

In [6]:
# calculate dendrogram
dendogram = g.community_edge_betweenness(directed=False, weights='weight')
# convert it into a flat clustering
clusters = dendogram.as_clustering()
# get the membership vector
eb_membership = clusters.membership
len(set(eb_membership))

4

The Girvan-Newman edge betweenness community detection algorithm finds fewer than the seven communities reported by Beveridge et al in the original paper. We could try a more computationally intensive method like optimal modularity, but this network is close to the limit of the number of vertices where this algorithm would apply. 

A heuristic method like `spinglass` will get a similar result for this type of undirected network in a fraction of the time, but will likely come up with a slightly different number of communities and membership than will `optimal_modularity`. See https://en.wikipedia.org/wiki/Modularity_(networks).

In [7]:
om_modules = g.community_optimal_modularity(weights='weight')
om_membership = om_modules.membership
len(set(om_membership))

7

In [9]:
# Represent groups as Strings to avoid a mapping bug in py2cytoscape.
g.vs['group'] = list(map(lambda x: str(x), om_membership))

### Centrality Metrics

In [86]:
g.vs['pagerank'] = g.pagerank(directed=False, weights='weight')

In [87]:
g.vs['betweenness'] = g.betweenness(directed=False, weights='weight')

In [88]:
def ordinal(n):
    """Crazy: https://stackoverflow.com/a/20007730"""
    return "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])


def ranks(entries):
    """Given a list of Numeric entries, returns a list of the ranks of each entry."""
    seq = sorted(entries)
    n = len(entries)
    return [ordinal(n - seq.index(x)) for x in entries]

In [89]:
g.vs['pr_rank'] = ranks(g.vs['pagerank'])
g.vs['bc_rank'] = ranks(g.vs['betweenness'])
g.vs['dc_rank'] = ranks(g.vs['indegree'])

In [90]:
g.vs[g.vs['pr_rank'].index("1st")]

igraph.Vertex(<igraph.Graph object at 0x111116138>, 98, {'name': 'Tyrion', 'imageurl': 'https://d1l1fon1fdqzt1.cloudfront.net/img/Tyrion.jpg', 'wiki': 'http://gameofthrones.wikia.com/wiki/Tyrion_Lannister', 'indegree': 36, 'outdegree': 36, 'label': 'Tyrion', 'group': '2', 'pagerank': 0.05545693845369458, 'betweenness': 1163.7833333333333, 'pr_rank': '1st', 'bc_rank': '2nd', 'dc_rank': '1st'})

## Visualize the network in Cytoscape

In [91]:
app.print_version()

{
  "apiVersion": "v1",
  "cytoscapeVersion": "3.5.1"
}


In [92]:
cy = CyRestClient()
cy.session.delete()

In [93]:
network = cy.network.create_from_igraph(g, name='storm of swords', collection='got')

## Customize the style

In [94]:
from sand.cytoscape.themes import colors as c
from sand.cytoscape.themes import label_positions as p

settings = {
    # node style
    'NODE_TRANSPARENCY': 255,
    'NODE_SIZE': 25,
    'NODE_BORDER_WIDTH': 4,
    'NODE_BORDER_PAINT': '#FFCC66',
    'NODE_FILL_COLOR': c.DARK_GREEN,
    'NODE_SELECTED_PAINT': c.BRIGHT_YELLOW,
    'NODE_SHAPE': 'RECTANGLE',

    # node label style
    'NODE_LABEL_COLOR': c.BRIGHT_GRAY,
    'NODE_LABEL_FONT_SIZE': 16,
    'NODE_LABEL_POSITION': p.LOWER_RIGHT,

    # edge style
    'EDGE_TRANSPARENCY': 255,
    'EDGE_WIDTH': 2.5,
    'EDGE_LINE_TYPE': 'SOLID',
    'EDGE_STROKE_SELECTED_PAINT': c.BRIGHT_YELLOW,
    'EDGE_STROKE_UNSELECTED_PAINT': c.BRIGHT_GRAY,
    'EDGE_TARGET_ARROW_UNSELECTED_PAINT': c.BRIGHT_GRAY,

    # network style
    'NETWORK_BACKGROUND_PAINT': c.DARK_GRAY
}

style_name = 'GoT'
style = cy.style.create(style_name)
style.update_defaults(settings)

In [95]:
app.lock_node_width_and_height(style_name, False)

True

In [96]:
# Map the label property in the igraph data to Cytoscape's NODE_LABEL visual property
style.create_passthrough_mapping(column='label', vp='NODE_LABEL', col_type='String')

In [97]:
# Add icons to vertices
style.create_passthrough_mapping(column='imageurl', vp='NODE_CUSTOMGRAPHICS_1', col_type='String')

In [98]:
# scale edges by weight
weight_to_width = su.create_slope(min=min(g.es['weight']), max=max(g.es['weight']), values=(1, 10))
style.create_continuous_mapping(column='weight', vp='EDGE_WIDTH', col_type='Double', points=weight_to_width)

In [99]:
# scale vertex size according to PageRank value.
pr_to_size = su.create_slope(min=min(g.vs['pagerank']), max=max(g.vs['pagerank']), values=(25, 125))
style.create_continuous_mapping(column='pagerank', vp='NODE_HEIGHT', col_type='Double', points=pr_to_size)
style.create_continuous_mapping(column='pagerank', vp='NODE_WIDTH', col_type='Double', points=pr_to_size)

In [100]:
# The size of its label corresponds to its betweenness centrality.
bc_to_size = su.create_slope(min=min(g.vs['betweenness']), max=max(g.vs['betweenness']), values=(14, 32))
style.create_continuous_mapping(column='betweenness', col_type='Double', vp='NODE_LABEL_FONT_SIZE', points=bc_to_size)

In [101]:
# Give each group a unique border color using the number of groups determined above.
border_colors = {
  '0': colors.BRIGHT_YELLOW,
  '1': colors.BRIGHT_RED,
  '2': colors.BRIGHT_BLUE,
  '3': colors.BRIGHT_GREEN,
  '4': colors.BRIGHT_ORANGE,
  '5': colors.BRIGHT_PURPLE,
  '6': colors.BRIGHT_WHITE
}

style.create_discrete_mapping(column='group', col_type='String', vp='NODE_BORDER_PAINT', mappings=border_colors)

In [102]:
cy.style.apply(style, network)

In [103]:
# Apply layout
cy.layout.apply(name='kamada-kawai', network=network)
cy.layout.fit(network)

In [104]:
# Load previous positions
positions_file = './data/storm_of_swords_positions.csv'
scp.layout_from_positions_csv(network, positions_file, cy)
cy.layout.fit(network)

### Save the positions of the vertices to support later updates

In [114]:
scp.positions_to_csv(network=network, path=positions_file)

### Export the view for use in cytoscape.js based visualizations

In [115]:
import json

net_view = network.get_first_view()
with open('./output/storm_of_swords.json', 'w') as out:
    json.dump(net_view, out, indent=2)

In [116]:
style_for_js = cy.style.get(style.get_name(), data_format='cytoscapejs')

**Correct the background images:**

In [117]:
def background_image(id, name):
    return {"selector": "#{}".format(id),
            "css": {
              "background-image": "/img/{}.jpg".format(name),
              "background-fit": "contain"
              }
            }


selectors = []
for v in net_view['elements']['nodes']:
    node = v['data']
    selectors.append(background_image(node['id'], node['name']))

In [118]:
style_for_js['style'] += selectors

**Add in elements we need to support zooming and fading:**

In [119]:
additions = [{
        "selector": "node.highlighted",
        "css": {
          "min-zoomed-font-size": 0,
          "z-index": 9999
        }
      },
      {
        "selector": "edge.highlighted",
        "css": {
          "opacity": 0.8,
          "width": 4,
          "z-index": 9999
        }
      },
      {
        "selector": ".faded",
        "css": {
          "events": "no",
          "background-image-opacity": 0.1
        }
      },
      {
        "selector": "node.faded",
        "css": {
          "opacity": 0.08
        }
      },
      {
        "selector": "edge.faded",
        "css": {
          "opacity": 0.06
        }
      },
      {
        "selector": ".hidden",
        "css": {
          "display": "none"
        }
      }
]

In [120]:
style_for_js['style'] += additions

**Fix an issue where cytoscape exports string format for doubles:**

In [121]:
string_bug_fix = [
    {'css': {'font-size': 32}, 
     'selector': 'node[betweenness > 1166.15]'},
    {'css': {'font-size': 32}, 
     'selector': 'node[betweenness = 1166.15]'},
    {'css': {'font-size': 'mapData(betweenness,0,1166.15,14,32)'},
     'selector': 'node[betweenness > 0][betweenness < 1166.15]'}]

In [122]:
style_for_js['style'] += string_bug_fix

**Write the final style**

In [123]:
with open('./output/got.style.json', 'w') as out:
    json.dump(style_for_js, out, indent=2)